<a href="https://colab.research.google.com/github/plutasnyy/recognizeeyebloodvessels/blob/master/recog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# You can change the directory name
LOG_DIR = 'tb_logs1y'

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)
  
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-04-27 23:29:38--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.173.32.212, 35.173.3.255, 52.45.248.161, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.173.32.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14991793 (14M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  14.30M  18.3MB/s    in 0.8s    

2019-04-27 23:29:39 (18.3 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [14991793/14991793]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://34ebb77f.ngrok.io


In [0]:
import logging
import os
import glob
import random

import keras
from keras import backend as K
from keras import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, Activation
from keras.utils import to_categorical
from keras.optimizers import SGD

import tensorflow as tf
from tensorflow.python.client import device_lib

from collections import Counter
from time import gmtime, strftime

from PIL import Image
from numpy import asarray
from skimage import transform, exposure
from skimage.filters import sobel

import sklearn
import cv2
import matplotlib.pyplot as plt
import numpy as np


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
logging.basicConfig(level=logging.DEBUG)

PATCH_SIZE = 48
HALF_OF_PATCH_SIZE = int(PATCH_SIZE / 2)
SPLIT_PATCHES_SIZE = 10000
LONG_EDGE_SIZE = 700


def correct_image(image):
    #TODO run this function only when process tensor, not during creating an object
    logging.info('Correct an image with shape: {}'.format(image.shape))
    bw_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    image_adapt = exposure.equalize_adapthist(bw_image)
    logarithmic_corrected = exposure.adjust_log(image_adapt, 1)
    return logarithmic_corrected
  
class Tensor:
    def __init__(self, base_image, vessels, mask, id):
        assert base_image.size == vessels.size, 'Images have different sizes'
        assert mask.size == vessels.size, 'Mask has wrong size'

        long_edge = max(base_image.size)
        scale = LONG_EDGE_SIZE / long_edge
        w, h = base_image.size
        c = len(base_image.getbands())
        w, h = int(w * scale), int(h * scale)
        logging.info(
            'Tensor resize from {} to {} with {} scale'.format((base_image.size, c), (w, h, c), round(scale, 2)))

        self.base_image = asarray(base_image.resize((w, h), resample=Image.NEAREST))  # 0-255
        self.corrected = (correct_image(
            self.base_image) / 255).astype(int)  # 0-1 TODO VERY BAD DEPENDENCY it should be moved out from this class
        self.vessels = (asarray(vessels.resize((w, h), resample=Image.NEAREST)) / 255).astype(
            int)  # 0-1

        if len(self.vessels.shape) == 3:
            self.vessels = self.vessels[:, :, 1]

        self.mask = (asarray(mask.resize((w, h))) / 255).astype(int)  # 0-1
        self.id = id

    def draw_tensor(self):
        """
        TODO This is very bad dependency to other functionality, this class should be independent, it was created only for tests and it should be removed when will be unused
        """
        from image_processor import draw_images
        draw_images([self.base_image, self.corrected, self.vessels, self.mask])

    def __repr__(self):
        return '{}: base_image: {}'.format(self.id, self.base_image.shape)


def create_tensor_from_file():
    images_path = sorted(glob.glob('/content/recognizeeyebloodvessels/train/*'))
    print(images_path)
    for i in range(0, len(images_path), 3):
        file_name = images_path[i].replace('/', '.').split('.')[1]
        logging.info('Process filepath: {}, fileName: {}, i: {}'.format(images_path[i], file_name, i))
        base_image = Image.open(images_path[i])
        mask = Image.open(images_path[i + 1])
        vessels = Image.open(images_path[i + 2])
        yield Tensor(base_image, vessels, mask, i)

def create_samples_from_tensor(tensor: Tensor):
    logging.info('Create samples from tensor: {}'.format(tensor))
    X, Y = list(), list()
    for (x, y), value in np.ndenumerate(tensor.mask):
        if x + PATCH_SIZE <= tensor.corrected.shape[0] and y + PATCH_SIZE <= tensor.corrected.shape[1]:
            center_x, center_y = x + HALF_OF_PATCH_SIZE, y + HALF_OF_PATCH_SIZE
            if tensor.mask[center_x][center_y] == 1:
                X.append(tensor.corrected[x: x + PATCH_SIZE, y: y + PATCH_SIZE])
                Y.append(tensor.vessels[center_x][center_y])
    return X, Y


def random_undersampling(X, y):
    """
    In this moment we will lose order of samples
    """
    minority_value, majority_value = 1, 0
    new_X, new_y = list(), list()
    length = len(y)
    quantity_of_minority = sum(y)
    quantity_of_majority = length - quantity_of_minority
    indexes_list = list(range(length))
    random.shuffle(indexes_list)
    skipped, to_skip = 0, quantity_of_majority - quantity_of_minority
    assert to_skip >= 0
    for index in indexes_list:
        if skipped < to_skip and y[index] == majority_value:
            skipped += 1
        else:
            new_X.append(X[index])
            new_y.append(y[index])

    result_X, result_Y = sklearn.utils.shuffle(new_X, new_y, random_state=0)
    return result_X, result_Y


def draw_images(images: list):
    logging.info('Draw {} images'.format(len(images)))
    size = np.ceil(np.sqrt(len(images)))
    fig = plt.figure(figsize=(32, 32))
    for i, img in enumerate(images):
        fig.add_subplot(size, size, i + 1)
        plt.imshow(img)
    plt.show()


def draw_grey_image(image):
    plt.imshow(image, cmap='gray')
    plt.show()


def preprocess_image(tensor: Tensor):
    logging.info('Started preprocess a tensor: {}'.format(tensor))
    edge_sobel = sobel(tensor.corrected)
    normalized_image = cv2.normalize(edge_sobel, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    _, thresholded_image = cv2.threshold(normalized_image, 20, 255, cv2.THRESH_BINARY)
    cleaned_image = (thresholded_image * tensor.mask).astype(int)
    return np.invert(cleaned_image)

class Model:
    def __init__(self):
        self.model = self._build_model()
        filepath = 'new_best.hdf5'
        self.tb_call_back = keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=True,
                                                        write_images=True)
        self.checkpoint = ModelCheckpoint(filepath, save_weights_only=False, monitor='val_acc', verbose=0,
                                          save_best_only=True, mode='max')

    def load_weights(self, filename):
        logging.info('Load weights: {}'.format(filename))
        self.model.load_weights(filename)

    def fit(self, X, y):
        self.model.fit(X, y, validation_split=0.1, epochs=150, batch_size=32,
                       callbacks=[self.tb_call_back, self.checkpoint], verbose=1)

    def compile(self):
        # adam = keras.optimizers.Adam(lr=1e-5, epsilon=None, decay=0.0, amsgrad=False)
        sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=False)
        self.model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        logging.info('Compiled keras model')

    def evaluate(self, X, y):
        result = self.model.evaluate(X, y)
        logging.info('Model evaluated, loss: {} acc: {}'.format(result[0], result[1]))
        return result

    def predict(self, X):
        input = X
        if len(input.shape) == 2:
            input = input.reshape(1, PATCH_SIZE, PATCH_SIZE, 1)
            logging.debug('Reshaped before prediction')
        result = self.model.predict(input)
        logging.debug('Predicted: {}, return {}'.format(result, np.argmax(result)))
        logging.debug('For: {}'.format(input))
        return result

    @staticmethod
    def _build_model():
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3),
                         activation='relu',
                         input_shape=(PATCH_SIZE, PATCH_SIZE, 1)))
        model.add(Dropout(0.2))
        model.add(Conv2D(32, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(Dropout(0.2))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

#         model.add(Conv2D(128, (3, 3), activation='relu'))
#         model.add(BatchNormalization())
#         model.add(Activation("relu"))
#         model.add(Conv2D(128, (3, 3), activation='relu'))

        model.add(Flatten())
#         model.add(Dense(256, activation='relu'))
#         model.add(Dropout(0.2))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(2, activation='softmax'))

        logging.info('Created keras model')
        return model
      
model = Model()
# model.load_weights('best.hdf5')
model.compile()

for tensor in create_tensor_from_file():
    X, y = create_samples_from_tensor(tensor)

    logging.info('Patches were created')
    print(len(X),len(y))
    logging.info('Original dataset shape {}'.format(Counter(y)))
    X, y = random_undersampling(X, y)

    logging.debug('Resampled dataset shape {}'.format(Counter(y)))

    for start_index in range(0, len(X), SPLIT_PATCHES_SIZE):
        end_index = min(start_index + SPLIT_PATCHES_SIZE, len(X))  # tricky way to avoid OutOfIndexError
        logging.info('Splitting set. Range: {}:{} Progress of this tensor: {}% Time: {}'.format(
            start_index, end_index, round(start_index / len(X) * 100), strftime("%Y-%m-%d %H:%M:%S", gmtime())))
        X_subset, y_subset = X[start_index:end_index], y[start_index:end_index]
        logging.debug('Cut dataset result countered shape {}'.format(Counter(y_subset)))

        X_subset = np.array(X_subset).reshape(len(X_subset), PATCH_SIZE, PATCH_SIZE, 1)
        y_subset = to_categorical(y_subset)
        logging.debug('Shape X: {}, y: {}'.format(X_subset.shape, y_subset.shape))

        model.fit(X_subset, y_subset)
        break
    break


INFO:root:Created keras model
INFO:root:Compiled keras model
INFO:root:Process filepath: /content/recognizeeyebloodvessels/train/001.JPG, fileName: content, i: 0
INFO:root:Tensor resize from ((3504, 2336), 3) to (700, 466, 3) with 0.2 scale
INFO:root:Correct an image with shape: (466, 700, 3)


['/content/recognizeeyebloodvessels/train/001.JPG', '/content/recognizeeyebloodvessels/train/001_mask.jpg', '/content/recognizeeyebloodvessels/train/002.tif', '/content/recognizeeyebloodvessels/train/003.jpg', '/content/recognizeeyebloodvessels/train/003_mask.jpg', '/content/recognizeeyebloodvessels/train/004.tif', '/content/recognizeeyebloodvessels/train/005.jpg', '/content/recognizeeyebloodvessels/train/005_mask.jpg', '/content/recognizeeyebloodvessels/train/006.tif', '/content/recognizeeyebloodvessels/train/007.JPG', '/content/recognizeeyebloodvessels/train/007_mask.jpg', '/content/recognizeeyebloodvessels/train/008.tif', '/content/recognizeeyebloodvessels/train/009.jpg', '/content/recognizeeyebloodvessels/train/009_mask.jpg', '/content/recognizeeyebloodvessels/train/010.tif', '/content/recognizeeyebloodvessels/train/011.jpg', '/content/recognizeeyebloodvessels/train/011_mask.jpg', '/content/recognizeeyebloodvessels/train/012.tif', '/content/recognizeeyebloodvessels/train/013.JPG', 

INFO:root:Create samples from tensor: 0: base_image: (466, 700, 3)
INFO:root:Patches were created
INFO:root:Original dataset shape Counter({0: 237936, 1: 14812})


252748 252748


DEBUG:root:Resampled dataset shape Counter({1: 14812, 0: 14812})
INFO:root:Splitting set. Range: 0:10000 Progress of this tensor: 0% Time: 2019-04-27 23:32:25
DEBUG:root:Cut dataset result countered shape Counter({0: 5027, 1: 4973})
DEBUG:root:Shape X: (10000, 48, 48, 1), y: (10000, 2)


Train on 9000 samples, validate on 1000 samples
Epoch 1/150
9000/9000 [==============================] - 3s 328us/step - loss: 0.6932 - acc: 0.4988 - val_loss: 0.6933 - val_acc: 0.4920
Epoch 2/150
9000/9000 [==============================] - 3s 297us/step - loss: 0.6932 - acc: 0.5039 - val_loss: 0.6932 - val_acc: 0.4920
Epoch 3/150
9000/9000 [==============================] - 3s 296us/step - loss: 0.6932 - acc: 0.5012 - val_loss: 0.6933 - val_acc: 0.4920
Epoch 4/150
9000/9000 [==============================] - 3s 297us/step - loss: 0.6932 - acc: 0.5039 - val_loss: 0.6933 - val_acc: 0.4920
Epoch 5/150
9000/9000 [==============================] - 3s 298us/step - loss: 0.6932 - acc: 0.4963 - val_loss: 0.6934 - val_acc: 0.4920
Epoch 6/150
9000/9000 [==============================] - 3s 302us/step - loss: 0.6932 - acc: 0.5039 - val_loss: 0.6933 - val_acc: 0.4920
Epoch 7/150
9000/9000 [==============================] - 3s 314us/step - loss: 0.6932 - acc: 0.5039 - val_loss: 0.6932 - val_acc: 